In [1]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
from send_telegram import enviar_mensagem_telegram, formatar_resultado
from fh_filter import get_filter
from fh_live_data import fh_live_data
from fh_id_predict import over_under
import os
from supabase import create_client, Client
import math
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [2]:
def filter_min(over_under_resp, min_percent=70, min_odd=1.2):

    dict_gols_over = over_under_resp[1]['gols'][0]
    dict_gols_under = over_under_resp[1]['gols'][1]
    dict_corners_over = over_under_resp[2]['corners'][0]
    dict_corners_under = over_under_resp[2]['corners'][1]
    dict_info = over_under_resp[0]
    list_gols_over_resp = []
    list_gols_under_resp = []
    list_corners_over_resp = []
    list_corners_under_resp = []
    dict_resposta = {}

    for key, value in dict_gols_over.items():
        if (value[2] >= min_percent) and (value[3] >= min_odd):
            list_gols_over_resp.append(dict_gols_over[key])


    for key, value in dict_corners_under.items():
        if (value[2] >= min_percent) and (value[3] >= min_odd):
            list_corners_under_resp.append(dict_corners_under[key])

   
    for key, value in dict_corners_over.items():
        if (value[2] >= min_percent) and (value[3] >= min_odd):
            list_corners_over_resp.append(dict_corners_over[key])


    for key, value in dict_corners_under.items():
        if (value[2] >= min_percent) and (value[3] >= min_odd):
            list_corners_under_resp.append(dict_corners_under[key])

    if len(list_gols_over_resp) + len(list_gols_under_resp) + len(list_corners_over_resp) + len(list_corners_under_resp) > 0:
        dict_resposta['info'] = dict_info

    if len(list_gols_over_resp) > 0:
        dict_resposta['gols_over'] = list_gols_over_resp
    if len(list_gols_under_resp) > 0:
        dict_resposta['gols_under'] = list_gols_under_resp

    
    if len(list_corners_over_resp) > 0:
        dict_resposta['corners_over'] = list_corners_over_resp
    if len(list_corners_under_resp) > 0:
        dict_resposta['corners_under'] = list_corners_under_resp

    


    return dict_resposta

def resposts_live(list_ids_live):
    list_resposts = []
    for id_live in list_ids_live:
        print(f'Processing live id: {id_live}')
        over_under_resp = over_under(id_live) 
        list_resposts.append(filter_min(over_under_resp))

    return list_resposts

In [3]:
list_ids_live = fh_live_data()

14131963


In [4]:
list_resposts_live = resposts_live(list_ids_live)
list_resposts_live

Processing live id: 14131963


[{'info': {'event_id': 14131963,
   'id_tournament': 'Liga Portugal Betclic',
   'home_name': 'FC Porto',
   'away_name': 'Estoril Praia',
   'home_score': 1,
   'awayScore': 0,
   'home_statistics': {'Ball possession': 46,
    'Expected goals': 0.23,
    'Total shots': 2,
    'Shots on target': 2,
    'Shots inside box': 2,
    'Final third entries': 10,
    'cornerKicks': 1,
    'fouls': 11,
    'yellowCards': 1},
   'away_statistics': {'Ball possession': 54,
    'Expected goals': 0.99,
    'Total shots': 9,
    'Shots on target': 2,
    'Shots inside box': 5,
    'Final third entries': 28,
    'cornerKicks': 1,
    'fouls': 6,
    'yellowCards': 2}},
  'corners_over': [[5, 99, 83.19327731092437, 1.202020202020202]]}]

In [5]:
mensagem = formatar_resultado(list_resposts_live)
display(mensagem)

'<b>Previsão 1:</b>\nPartida: FC Porto vs Estoril Praia\nTorneio: Liga Portugal Betclic\nCorners Over: 1 previsões\n- Over 5: Probabilidade 83.19%, Odd 1.20\n'

In [6]:

enviar_mensagem_telegram(mensagem)

Mensagem enviada com sucesso via Telegram.


True